In [1]:
import pandas as pd
import constants as cs
import numpy as np
import matplotlib.pyplot as plt
import zipfile
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, make_scorer
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.neural_network import MLPRegressor
import joblib
from sklearn.ensemble import VotingRegressor

In [2]:
def load_data(filename: str) -> pd.DataFrame:
    with zipfile.ZipFile(f"../{filename}.zip") as data_zip:
        with data_zip.open(f"{filename}.csv") as developer_data:
            return pd.read_csv(developer_data)


In [61]:
def custom_accuracy_score(y_true, y_pred):
        accuracy_count = 0
        y_true = np.array(y_true, dtype='float64')
        for true_val, pred_val in zip(y_true, y_pred):
            if abs(true_val - pred_val) <= 20000:
                accuracy_count += 1

        accuracy = accuracy_count / len(y_true)
        return accuracy

In [62]:
def mean_absolute_error(y_true, y_pred):
    absolute_errors = np.abs(y_true - y_pred)
    mae = np.mean(absolute_errors)
    return mae

In [63]:
def modelPredictAndScore(X_valid, y_valid, model, modelName):
    predictions = model.predict(X_valid)
    custom_score_valid = custom_accuracy_score(y_valid[cs.COMPENSATION], predictions)
    rmse_valid = mean_absolute_error(y_valid[cs.COMPENSATION], predictions)

    print(f"{modelName} accuracy is:{custom_score_valid}")
    print(f"{modelName} root mean square is:{rmse_valid}")



In [64]:
na_data = load_data(cs.NA_TRAIN_DATA)
X_valid = load_data(cs.X_VALID)
y_valid = load_data(cs.Y_VALID)
print(y_valid.head(5))

scaler_model = joblib.load(f'{cs.ML_MODELS_FOLDER}/scaler.pkl')
svr_model = joblib.load(f'{cs.ML_MODELS_FOLDER}/{cs.BEST_SVR_MODELS}.pkl')
rf_model = joblib.load(f'{cs.ML_MODELS_FOLDER}/{cs.BEST_RF_MODELS}.pkl')
gb_model = joblib.load(f'{cs.ML_MODELS_FOLDER}/{cs.BEST_GB_MODELS}.pkl')
xgboost_model = joblib.load(f'{cs.ML_MODELS_FOLDER}/{cs.BEST_XGBOOST_MODELS}.pkl')

modelPredictAndScore(X_valid, y_valid, rf_model, cs.RANDOM_FOREST_MODELS)
X_valid_scaled = scaler_model.transform(X_valid)
modelPredictAndScore(X_valid_scaled, y_valid, svr_model, cs.SVR_MODELS)
modelPredictAndScore(X_valid, y_valid, gb_model, cs.GB_MODELS)
modelPredictAndScore(X_valid, y_valid, xgboost_model, cs.XGBOOST_MODELS)

       CompTotal
0   97576.939145
1  130000.000000
2   52541.428770
3   90071.020749
4   76560.367637
Random_forest_models accuracy is:0.3864528795811518
Random_forest_models root mean square is:45493.94137171258
SVR_models accuracy is:0.41590314136125656
SVR_models root mean square is:43633.06867091712
GB_models accuracy is:0.39037958115183247
GB_models root mean square is:47821.33329153133
XGBoost_models accuracy is:0.3949607329842932
XGBoost_models root mean square is:48537.84579108745


In [65]:
voting_regressor = VotingRegressor(
    estimators=[('rf', rf_model), ('gb', gb_model), ('xgboost', xgboost_model), ('svr', svr_model)]
)

In [66]:
X_train = load_data('X_train_data')
y_train = load_data('y_train_data')
y_train = y_train.values.ravel()
voting_regressor.fit(X_train, y_train)

VotingRegressor(estimators=[('rf',
                             RandomForestRegressor(max_depth=125,
                                                   min_samples_leaf=10,
                                                   n_estimators=150)),
                            ('gb',
                             GradientBoostingRegressor(learning_rate=0.05,
                                                       max_depth=5)),
                            ('xgboost',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None,
                                          early_stopping_rounds=None,
                                          enable_categor...
                                          interaction_constraints=None,
       

In [67]:
modelPredictAndScore(X_valid, y_valid, voting_regressor, "Voting Classifier")

Voting Classifier accuracy is:0.44044502617801046
Voting Classifier root mean square is:36626.847748986496
